# Installations

In [ ]:
%pip install pydicom matplotlib rt_utils scikit-learn scikit-image torch pandas

In [19]:
import pandas as pd
import matplotlib.pyplot as plt

# import required packages
import pydicom # pydicom is a package for working with DICOM files such as medical images, reports, and radiotherapy objects
import os # os module provides functions for interacting with the operating system
import matplotlib.pyplot as plt # matplotlib is a plotting library for the Python programming language and its numerical mathematics extension NumPy
import numpy as np # NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
from rt_utils import RTStructBuilder # rt_utils is a package for working with DICOM RT structure sets
import re # re module provides regular expression matching operations
from sklearn.model_selection import train_test_split # sklearn.model_selection is a module for splitting arrays or matrices into random train and test subsets
import numpy as np # for working with arrays
from skimage.transform import resize # for resizing images
import copy # for copying objects
import torch 
import cv2 # for working with images
import shutil # for working with files and collections of files

# Data Inspection

In [78]:
metadata = pd.read_csv('metadata.csv') # this loads the csv file into metadata dataframe

In [18]:
# return all the values in Series Description column
print(metadata['Series Description'].unique())

['RTstructT1' 'RTstructT2FS' 'RTstructAlignedT1toPET'
 'RTstructAlignedT2FStoPET' 'AlignedT1toPETBOX' 'RTstructPET' 'PET AC'
 'AXIAL SE T2 FAT SAT - RESEARCH' 'RTstructCT' 'AlignedT2FStoPETBOX'
 'AllignedT1toPETBOX' 'RTstructAlignedSTIRtoPET' 'AXIAL SE T1 - RESEARCH'
 'AlignedSTIRtoPETBOX' 'WB2DAC' 'AXT1' 'RTstructSTIR' 'STIR longTE AX'
 'PED2DAC' 'AX STIR' 'AX T1' 'T2ST RT - RESEARCH' 'T1 RT - RESEARCH'
 'AXIALT1' 'RTstructAlignedTtoPET' 'AX. T2 FSE  FS - RESEARCH'
 'AX. T1 SE - RESEARCH' 'Axial T1 - RESEARCH' 'PET AC 21'
 'T1 AX - RESEARCH' 'STIR AX - RESEARCH' 'CT IMAGES - RESEARCH' 'LEGS2DAC'
 'Axial T1 UPPER - RESEARCH' 'Axial FSET2 FS UPPER'
 '2. AXIAL T1 BOTH LEGS - RESEARCH' '3. AXIAL STIR - RESEARCH'
 'sag FSET2 Fatsat' 'Sagittal T1' '4. AXIAL  T1' '5. AXIAL T2 F.S.'
 'StandardFull' 'RTstructAlignedT1toPETBOX' 'RTstructAlignedSTIRtoPETBOX'
 'T2ST LT - RESEARCH' 'T1 LT - RESEARCH' 'CT IMAGES - LEGS - RESEARCH'
 'AXIAL FRFSE T2 Fatsat - RESEARCH' 'AXIAL FSE T1 - RESEARCH' 'O AX 

In [ ]:
metadata.head() # Show the first 5 rows of the metadata dataframe

In [ ]:
# Data Inspection
total_images = metadata['Number of Images'].sum()
# total number of series is the number of rows in the metadata dataframe 
total_series = metadata.shape[0]

series_per_subject = metadata['Subject ID'].value_counts()
# sort in ascending order of subject ID
series_per_subject = series_per_subject.sort_index()

series_per_modality = metadata['Modality'].value_counts()
images_per_subject = metadata.groupby('Subject ID')['Number of Images'].sum()
# sort in ascending order of subject ID
images_per_subject = images_per_subject.sort_index()

images_per_modality = metadata.groupby('Modality')['Number of Images'].sum()

print('Total number of images: ', total_images)
print('Total number of series: ', total_series)
print('Number of series per subject: \n', series_per_subject)
print('Number of series per modality: \n', series_per_modality)
print('Number of images per subject: \n', images_per_subject)
print('Number of images per modality: \n', images_per_modality)

In [ ]:
# Histogram of number of series UID for each subject ID
plt.figure(figsize=(10, 6))
series_per_subject.plot(kind='bar')
plt.title('Number of Series UID for Each Subject ID')
plt.xlabel('Subject ID')
plt.ylabel('Number of Series UID')
plt.show()

# Histogram of number of images for each subject ID
plt.figure(figsize=(10, 6))
images_per_subject.plot(kind='bar')
plt.title('Number of Images for Each Subject ID')
plt.xlabel('Subject ID')
plt.ylabel('Number of Images')
plt.show()

# Histogram of number of images for each modality
plt.figure(figsize=(10, 6))
images_per_modality.plot(kind='bar')
plt.title('Number of Images for Each Modality')
plt.xlabel('Modality')
plt.ylabel('Number of Images')
plt.show()

# Histogram of number of Series UID for each modality
plt.figure(figsize=(10, 6))
series_per_modality.plot(kind='bar')
plt.title('Number of Series UID for Each Modality')
plt.xlabel('Modality')
plt.ylabel('Number of Series UID')
plt.show()


# Data Processing

This cell replaces the series description with "STIR", "T1toPET", "T2FStoPET", "CT", "PET", "T1", "T2", "DAC" for easier pairing of RTStruct with its corresponding images

In [79]:
# Define a function to modify the series description
def modify_series_description(series_description):
    if "STIRtoPET" in series_description:
        return "STIRtoPET"
    elif "STIR" in series_description:
        return "STIR"
    elif "Stir" in series_description:
        return "STIR"
    elif "AX IR" in series_description:
        return "STIR"
    elif "T1toPET" in series_description:
        return "T1toPET"
    elif "T2FStoPET" in series_description:
        return "T2FStoPET"
    elif "TtoPET" in series_description:
        return "T1toPET"
    elif "PET" in series_description:
        return "PET"
    elif "T1" in series_description:
        return "T1"
    elif "T2FS" in series_description:
        return "T2FS"
    elif "T2 F" in series_description:
        return "T2FS"
    elif "FSE T2" in series_description:
        return "T2FS"
    elif "T2SP" in series_description:
        return "T2FS"
    elif "FAT T2" in series_description:
        return "T2FS"
    elif "T2" in series_description:
        return "STIR"
    elif "CT" in series_description:
        return "CT"
    elif "DAC" in series_description:
        return "PET"
    elif "Standard" in series_description:
        return "CT"
    else:
        return series_description  # If no match, return the original series description

# Apply the function to the 'Series Description' column
metadata['Series Description'] = metadata['Series Description'].apply(modify_series_description)

This cell filters the metadata dataframe into its corresponding modalities (MR, CT, PT), and also concatenates the corresponding RTstructs to it

In [80]:
# only include Series UID, Subject ID, Study UID, Series Description, Modality, and File Location of metadata
metadata_filtered = metadata[['Series UID', 'Subject ID', 'Study UID', 'Series Description', 'Modality', 'File Location']]

# Separate the dataframe based on the 'Modality' column
df_MR = metadata_filtered[metadata_filtered['Modality'] == 'MR']
df_CT = metadata_filtered[metadata_filtered['Modality'] == 'CT']
df_PT = metadata_filtered[metadata_filtered['Modality'] == 'PT']
df_RTSTRUCT = metadata_filtered[metadata_filtered['Modality'] == 'RTSTRUCT']

# Filter RTSTRUCT rows where Study UID has a match in MR, CT, and PT dataframes and a match in Series Description
rtstruct_mr = df_RTSTRUCT[df_RTSTRUCT['Study UID'].isin(df_MR['Study UID'])]
rtstruct_mr = rtstruct_mr[rtstruct_mr['Series Description'].isin(df_MR['Series Description'])]
rtstruct_ct = df_RTSTRUCT[df_RTSTRUCT['Study UID'].isin(df_CT['Study UID'])]
rtstruct_ct = rtstruct_ct[rtstruct_ct['Series Description'].isin(df_CT['Series Description'])]
rtstruct_pt = df_RTSTRUCT[df_RTSTRUCT['Study UID'].isin(df_PT['Study UID'])]
rtstruct_pt = rtstruct_pt[rtstruct_pt['Series Description'].isin(df_PT['Series Description'])]

# Append these RTSTRUCT rows to the corresponding dataframes
df_MR_rtstruct = pd.concat([df_MR, rtstruct_mr])
df_CT_rtstruct = pd.concat([df_CT, rtstruct_ct])
df_PT_rtstruct = pd.concat([df_PT, rtstruct_pt])

Now, for each of df_MR_rtstruct, df_CT_rtstruct, df_PT_rtstruct dataframes, we group them according to their Subject ID into a suitable dictionary

In [81]:
# Group each dataframe by 'Subject ID'
df_MR_grouped = df_MR_rtstruct.groupby('Subject ID')
df_CT_grouped = df_CT_rtstruct.groupby('Subject ID')
df_PT_grouped = df_PT_rtstruct.groupby('Subject ID')

# Create a dictionary where each key is a 'Subject ID' and each value is a subset of the dataframe for that subject
df_MR_dict = {group: df for group, df in df_MR_grouped}
df_CT_dict = {group: df for group, df in df_CT_grouped}
df_PT_dict = {group: df for group, df in df_PT_grouped}

# Dataset Preprocessing and Serialisation

Functions

In [39]:
# this function will load the dicom images and mask from the given directory
def load_data(dicom_dir, rtstruct_dir):
    rtstruct_dir = os.path.join(rtstruct_dir, '1-1.dcm')
    rtstruct = RTStructBuilder.create_from(
    dicom_series_path=dicom_dir, 
    rt_struct_path= rtstruct_dir
    )  # Load existing RT Struct. Requires the series path and existing RT Struct path

    # print(rtstruct.get_roi_names()) # view all of the ROI names from within the image

    mask_3d = rtstruct.get_roi_mask_by_name("GTV_Mass") # loading the 3D Mask from within the RT Struct

    image_files = [os.path.join(dicom_dir, f) for f in os.listdir(dicom_dir) if f.endswith('.dcm')] # get a list of all DICOM files in the directory
    image_files.sort(reverse=True) # sort the image_files by their name
    images = {pydicom.dcmread(f).SOPInstanceUID: pydicom.dcmread(f) for f in image_files} # load the DICOM images

    return images, mask_3d


# this function will display the images and mask
def view_data(images, mask_3d):
    for i in range(mask_3d.shape[2]):
        uid = list(images.keys())[i]  # get the SOPInstanceUID of the ith image


        mask_slice = mask_3d[:, :, i]

        # if the mask_slice dimensions are not the same as the image dimensions, crop the side with more pixels and pad the side with less pixels with zeros
        if mask_slice.shape != images[uid].pixel_array.shape:
            if mask_slice.shape[0] > images[uid].pixel_array.shape[0]:
                # crop the mask_slice
                mask_slice = mask_slice[:images[uid].pixel_array.shape[0], :]
            elif mask_slice.shape[0] < images[uid].pixel_array.shape[0]:
                # pad the mask_slice with zeros starting from the bottom
                mask_slice = np.pad(mask_slice, ((0, images[uid].pixel_array.shape[0] - mask_slice.shape[0]), (0, 0)), 'constant')

            if mask_slice.shape[1] > images[uid].pixel_array.shape[1]:
                # crop the mask_slice
                mask_slice = mask_slice[:, :images[uid].pixel_array.shape[1]]
            elif mask_slice.shape[1] < images[uid].pixel_array.shape[1]:
                # pad the mask_slice with zeros from the right
                mask_slice = np.pad(mask_slice, ((0, 0), (0, images[uid].pixel_array.shape[1] - mask_slice.shape[1])), 'constant')

        # Now that the mask_slice and image dimensions are the same, we can resize the dimensions of both mask_slice and image for machine learning
        mask_slice = cv2.resize(mask_slice.astype(np.uint8), (128, 128), interpolation=cv2.INTER_NEAREST)
        img_slice = cv2.resize(images[uid].pixel_array, (128, 128))

        plt.imshow(img_slice, cmap='gray') # plot the ith image
        plt.imshow(mask_slice, cmap='jet', alpha=0.5) # overlay the mask with transparency
        plt.axis('off')
        plt.show()


# this function will serialise the images and mask for machine learning
def serialise_data(images, mask_3d, img_directory, mask_directory):
    for i in range(mask_3d.shape[2]):
        uid = list(images.keys())[i]  # get the SOPInstanceUID of the ith image

        mask_slice = mask_3d[:, :, i]

        # if the mask_slice dimensions are not the same as the image dimensions, crop the side with more pixels and pad the side with less pixels with zeros
        if mask_slice.shape != images[uid].pixel_array.shape:
            if mask_slice.shape[0] > images[uid].pixel_array.shape[0]:
                # crop the mask_slice
                mask_slice = mask_slice[:images[uid].pixel_array.shape[0], :]
            elif mask_slice.shape[0] < images[uid].pixel_array.shape[0]:
                # pad the mask_slice with zeros starting from the bottom
                mask_slice = np.pad(mask_slice, ((0, images[uid].pixel_array.shape[0] - mask_slice.shape[0]), (0, 0)), 'constant')

            if mask_slice.shape[1] > images[uid].pixel_array.shape[1]:
                # crop the mask_slice
                mask_slice = mask_slice[:, :images[uid].pixel_array.shape[1]]
            elif mask_slice.shape[1] < images[uid].pixel_array.shape[1]:
                # pad the mask_slice with zeros from the right
                mask_slice = np.pad(mask_slice, ((0, 0), (0, images[uid].pixel_array.shape[1] - mask_slice.shape[1])), 'constant')

        # Now that the mask_slice and image dimensions are the same, we can resize the dimensions of both mask_slice and image for machine learning
        mask_slice = cv2.resize(mask_slice.astype(np.uint8), (128, 128), interpolation=cv2.INTER_NEAREST)
        img_slice = cv2.resize(images[uid].pixel_array, (128, 128), interpolation = cv2.INTER_LANCZOS4)
        
        # normalise the img_slice
        img_slice = img_slice / 255.0

        # serialise the img_slice into jpg format in the img_directory
        try:
            cv2.imwrite(os.path.join(img_directory, f'{images[uid].SeriesInstanceUID}_{i}.png'), img_slice)
        except Exception as e:
            print(f"Error saving image: {e}")
        
        # serialise the mask_slice into png format in the mask_directory
        try:
            cv2.imwrite(os.path.join(mask_directory, f'{images[uid].SeriesInstanceUID}_{i}.png'), mask_slice*255)
        except Exception as e:
            print(f"Error saving mask: {e}")


def split_data_into_train_val_test(dataset_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, random_state=42):
    # Define the sub-directories
    image_dir = os.path.join(dataset_dir, "train", "images")
    mask_dir = os.path.join(dataset_dir, "train", "mask")

    # Get the lists of images and masks
    images = os.listdir(image_dir)
    masks = os.listdir(mask_dir)

    # Split the datasets into training, validation and testing sets
    temp_images, test_images, temp_masks, test_masks = train_test_split(images, masks, test_size=test_ratio, random_state=random_state)
    train_images, val_images, train_masks, val_masks = train_test_split(temp_images, temp_masks, test_size=val_ratio/(train_ratio+val_ratio), random_state=random_state)

    # Define the destination directories
    test_image_dest_dir = os.path.join(dataset_dir, "test", "images")
    test_mask_dest_dir = os.path.join(dataset_dir, "test", "mask")
    val_image_dest_dir = os.path.join(dataset_dir, "val", "images")
    val_mask_dest_dir = os.path.join(dataset_dir, "val", "mask")

    # Move the testing images and masks to the testing directory
    for img in test_images:
        shutil.move(os.path.join(image_dir, img), test_image_dest_dir)

    for mask in test_masks:
        shutil.move(os.path.join(mask_dir, mask), test_mask_dest_dir)

    # Move the validation images and masks to the validation directory
    for img in val_images:
        shutil.move(os.path.join(image_dir, img), val_image_dest_dir)

    for mask in val_masks:
        shutil.move(os.path.join(mask_dir, mask), val_mask_dest_dir)


### Serialise

Here, we take all the MR images and mask from all patients (STS_001 to STS_051)

In [83]:
# dictionary for each of STS_001 to STS_051
data_dict = {}

# specify the path to the dataset
dataset_mr_path = 'dataset_mr'
img_path = os.path.join(dataset_mr_path, 'train', 'images')
mask_path = os.path.join(dataset_mr_path, 'train', 'mask')

# filter df_MR_dict for STS_001 to separate DCM images from RTSTRUCT
for key, value in df_MR_dict.items():
    img_df_MR_dict = value[value['Modality'] == 'MR']
    rtstruct_df_MR_dict = value[value['Modality'] == 'RTSTRUCT']
    
    # initialise array
    data_dict[key] = []
    match = False # for debugging

    # load the dcm and rtstruct data
    for index, row in value.iterrows(): # iterate through each row in the dataframe of the current patient (value)
        if row['Modality'] == 'MR': # if the current row is not an RTSTRUCT
            for index1, row1 in value[value['Series UID'] != row['Series UID']].iterrows(): # iterate through the remaining rows in the dataframe of the current patient (value)
                if row1['Series Description'] == row['Series Description']: # if the current row is the corresponding RTSTRUCT to the current MR image
                    data_dict[key].append(load_data(row['File Location'], row1['File Location'])) # load the data and append it to the array
                    match = True # for debugging
                    break # break out of the loop
            if match == False: # for debugging
                raise Exception('no match found for ' + row['Series UID']) # for debugging
            match = False # for debugging
    
    # serialise the data
    for i in range(len(data_dict[key])):
        serialise_data(data_dict[key][i][0], data_dict[key][i][1], img_path, mask_path)

In [84]:
split_data_into_train_val_test(dataset_mr_path)